In [3]:
import pandas as pd
import subprocess
import os

# Consensus vs TP data

In [6]:
# Load TP data
TP_data = pd.read_csv("./data/TP_SIDERs_coor.csv", sep=",", header=0)
print(TP_data.shape)
print(TP_data.dtypes)
TP_data.head()

(1606, 14)
qseqid       object
sseqid       object
pident      float64
length        int64
qstart        int64
qend          int64
sstart        int64
send          int64
evalue      float64
bitscore    float64
qlen          int64
slen          int64
sstrand      object
sseq         object
dtype: object


,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,1-112979r,LinJ.01,99.614,259,1,259,114017,113760,5.440000e-135,472.0,259,278267,minus,GGGGTGAGGCGGCGGCGCACAGACACACACACACACACACACACGC...
1,1-129494r,LinJ.01,100.000,447,1,447,130577,130131,0.000000e+00,826.0,447,278267,minus,TGATGACGGGGCGGCGGCGCACAGACACACACACACACACACACAC...
2,1-136422d,LinJ.01,100.000,461,1,461,137458,137918,0.000000e+00,852.0,461,278267,plus,TGACGACGGGGTGGGGCGGCGGCGCACAGACACACACACACACACA...
3,1-204526d,LinJ.01,100.000,432,1,432,205558,205989,0.000000e+00,798.0,432,278267,plus,GATGACGGGGTGGGGCGGCGGCGCACACACACACACGCACACACCT...
4,1-24471d,LinJ.01,100.000,660,1,660,24099,24758,0.000000e+00,1219.0,660,278267,plus,GGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATCCCAGG...


In [1]:
# Load consensus data
consensus_data = pd.read_csv("./data_software_output/consensus_data_seqs.csv", sep=",", header=0)
print(consensus_data.shape)
print(consensus_data.dtypes)
consensus_data.head()

NameError: name 'pd' is not defined

In [12]:
# Check if TP data "sstart" < "send". For that, see where is not
(TP_data[TP_data["sstart"] > TP_data["send"]]).sum()

qseqid      1-112979r1-129494r1-36305r1-55910r2-193840r2-2...
sseqid      LinJ.01LinJ.01LinJ.01LinJ.01LinJ.02LinJ.02LinJ...
pident                                              78392.551
length                                                 397871
qstart                                                    812
qend                                                   397832
sstart                                              444164165
send                                                443767403
evalue                                                    0.0
bitscore                                             726683.0
qlen                                                   397871
slen                                                958894346
sstrand     minusminusminusminusminusminusminusminusminusm...
sseq        GGGGTGAGGCGGCGGCGCACAGACACACACACACACACACACACGC...
dtype: object

In [8]:
# Group data by chromosomes
TP_data_grouped = TP_data.groupby("sseqid")
consensus_data_grouped = consensus_data.groupby("sseqid")

In [ ]:
path_1 = "./compare/consensus_vs_TP"
Compraison_dict_1 = {}
for (name1, group1), (name2, group2) in zip(TP_data_grouped, consensus_data_grouped):
    # group1 should be the TP data
    # ------------------------------------------------------------------------------
    path_chr = os.path.join(path_1, name1)
    os.makedirs(path_chr, exist_ok=True)
    # ------------------------------------------------------------------------------
    group1_total = group1[["sseqid", "sstart", "send"]].copy()
    group2_total = group2[["sseqid", "sstart", "send"]].copy()
    # ------------------------------------------------------------------------------
    group1_total.sort_values(by=["sstart", "send"], inplace=True)
    group2_total.sort_values(by=["sstart", "send"], inplace=True)
    # ------------------------------------------------------------------------------
    path_group1_total = os.path.join(path_chr, "group1_total.bed")
    path_group2_total = os.path.join(path_chr, "group2_total.bed")
    # ------------------------------------------------------------------------------
    group1_total.to_csv(path_group1_total, sep="\t", header=False, index=False)  # tabular sep because of bed format
    group2_total.to_csv(path_group2_total, sep="\t", header=False, index=False)  # tabular sep because of bed format

    # ------------------------------------------------------------------------------
    condition = 1
    cmd = f"bedops --element-of {condition} {path_group1_total} {path_group2_total}"
    result = subprocess.check_output(cmd, shell=True, universal_newlines=True)
    result_df = pd.DataFrame([x.split("\t") for x in result.split("\n") if x], 
                                columns=["sseqid", "sstart", "send"])
    result_df = columns_to_numeric(result_df, ["sstart", "send"])  # defined function
    comparison_dict_1[name1] = [f"{result_df.shape[0]}/{group1_total.shape[0]}", f"{result_df.shape[0]/group1_total.shape[0]*100:.2f}"]
comparison_dict_1